# Data Manipulation with pandas Interactive Notes

Review and practice the concepts and skills you learned in DataCamp's Data Manipulation with pandas course! This is an interactive notebook powered by DataCamp Workspace. 

_Note: Some later examples depend on code in earlier examples. To ensure variables and imports are available to you, click "Run All" in the top of this workspace._

## Chapter 1: DataFrames

### 1.1 Introducing DataFrames
`pandas` is an essential Python package for data manipulation. In `pandas`, rectangular data is represented as a DataFrame object.  Every value within a column has the same data type, but different columns can contain different data types.

In [ ]:
# Importing packages
import pandas as pd
import numpy as np

# Importing an advocado dataset as a DataFrame
avocado = pd.read_csv("data/avocado.csv")

When you get a new DataFrame to work with, the first thing you need to do is explore it and see what it contains. The following code cells show useful methods and attributes for this.

In [ ]:
# Return the first few rows of a DataFrame
avocado.head()

In [ ]:
# Compute some summary statistics for numerical columns
avocado.describe()

In [ ]:
# Return the number of rows followed by the number of columns
avocado.shape

In [ ]:
# Print the names of columns, the data types they contain, and whether they have any missing values
avocado.info()

DataFrames consist of three different components, accessible using attributes.

In [ ]:
# Return the data values in a 2D NumPy array
avocado.values

In [ ]:
# Return the row labels
avocado.index

In [ ]:
# Return the column names
avocado.columns

### 1.2 Sorting and subsetting

You can sort the rows of a DataFrame using the `.sort_values()` method, passing in column name(s) that you want to sort by.

In [ ]:
# Sort in ascending order
avocado.sort_values("date")

By default, data will be sorted in ascending order, but you can use the `ascending` argument to change that. If you want to sort by multiple columns, you can pass a list to `ascending` to specify which columns you want in asending or descending order.

In [ ]:
# Sort in descending order
avocado.sort_values("date", ascending=False)

# Sort by multiple variables by passing lists of column names and booleans
avocado.sort_values(["nb_sold", "date", "type"], ascending=[False, False, True])

You can subset one or more **columns** from a DataFrame using square brackets (`[]`).

In [ ]:
# Subset one column
avocado["type"]

# Subset multiple columns
avocado[["date", "avg_price"]]

The most common way to subset **rows** is by creating logical conditions to filter against.

In [ ]:
# Subset based a numberic threshold
avocado[avocado["avg_price"] < 0.90]

# Subset based on text data
avocado[avocado["size"] == "small"]

# Subset based on dates
avocado[avocado["date"] > "2018-01-01"]

# Subset based on multiple conditions
avocado[(avocado["size"] == "small") & (avocado["date"] > "2018-01-01")]

The `.isin()` method is the easiest way to filter on multiple values of a categorical variable:

In [ ]:
# Filter for only large and extra large avocados
large_sizes = avocado["size"].isin(["large", "extra_large"])
avocado[large_sizes]

### 1.3 New columns

Sometimes, you may have to add new columns to your DataFrame derived from existing columns. You can do this by defining a new column name and the calculation required.

In [ ]:
# Create a new column for average price in Euros (assuming 1 USD = 0.9 Euro)
avocado["avg_price_euros"] = avocado["avg_price"] * 0.9

# Create a new column for total sales
avocado["total_sales"] = avocado["avg_price"] * avocado["nb_sold"]

# View the DataFrame to see two new columns!
avocado

## Chapter 2: Aggregating DataFrames
### 2.1 Summary statistics

There are lots of summary statistics that you can compute on columns (e.g. `.median()`, `.mode()`, `min()`, `max()`, `var()`, `std()`, `sum()`, `cumsum()`).

In [ ]:
# You can use .max() and .min() on dates
avocado["date"].max()

In [ ]:
# .cumsum() returns a number for each row of the DataFrame
avocado["nb_sold"].cumsum()

Sometimes, you may need a different function to summarize data. The `.agg()` method allows you to apply your own custom functions to a DataFrame, as well as apply functions to more than one column of a DataFrame at once. 

In [ ]:
# Define a custom function: this one computes the inter-quartile range (IQR)
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

# Apply the function to one column
avocado["avg_price"].agg(iqr)

# Apply the function to multiple columns using double-brackets
avocado[["avg_price", "nb_sold"]].agg(iqr)

# Apply multiple functions
avocado[["avg_price", "nb_sold"]].agg([iqr, np.median])

### 2.2 Counting
Dropping duplicates is often an important step to avoid counting double. You can drop duplicates using the `.drop_duplicates()` method. It takes an optional argument, `subset`, which are the column(s) you want to find duplicates based on.

In [ ]:
# Drop rows with duplicate dates
avocado.drop_duplicates(subset="date")

# Drop rows with duplicate date/type combinations
avocado.drop_duplicates(subset=["date", "type"])

The `.value_counts()` method is used to count categorical variables with two useful optional arguments:
- The `normalize` argument can be used to turn the counts into proportions of the total.
- Set the `sort` argument to `True` to sort the counts.

In [ ]:
# Count the number of each type of avocado
avocado["type"].value_counts()

# Get the proportion of each type of avocado
avocado["type"].value_counts(normalize=True)

# Count the number of rows for each year and sort
avocado["year"].value_counts(sort=True)

### 2.3 Grouped summary statistics

The previous section calculated summary statistics for all rows of a dataset. It's also useful to compare summary statistics of different groups within a dataset. The `.groupby()` method allows you to group the DataFrame by one or more columns. You can then subset the DataFrame to aggregate selected columns and use one or more aggregation functions.

In [ ]:
# You can group by type and get the average amount sold
avocado.groupby("type")["nb_sold"].mean()

# You can group by multiple columns and summarize multiple columns
avocado.groupby(["type", "size"])[["avg_price", "nb_sold"]].mean()

# You can apply multiple functions with .agg()
avocado.groupby(["type", "size"])[["avg_price", "nb_sold"]].agg([np.mean, np.median])

### 2.4 Pivot tables

Pivot tables are another way of calculating grouped summary statistics. The `.pivot_table()` method has:
- a `values` argument which takes the name of the column you want to summarize.
- an `index` argument which takes the name of the column you want to group by.
- an `aggfunc` argument which takes in a list of functions to summarize the `values`. By default, `.pivot_table()` uses the mean. 
- a  `columns` argument which takes in the name of any other columns you want to group by.
- a `fill_value` argument to define what should replace missing values. 
- a `margins` argument. Setting this to `True` enables summary statistics for multiple levels of the dataset.

In [ ]:
avocado.pivot_table(
    values="avg_price",
    index="year",
    aggfunc=[np.mean, np.max],
    columns="size",
    margins=True,
)

## Chapter 3: Slicing and indexing DataFrames
### 3.1 Explicit indexes

You can set a column as the index using the `.set_index("column_name")` method. 

_Note: Index values don't need to be unique._

In [ ]:
temperatures = pd.read_csv("data/temperatures.csv")

# Index temperatures by city
temps_city = temperatures.set_index("city")

You can reset an index using the `.reset_index()` method. You can set the `drop` argument as `True` to discard the former index completely.

In [ ]:
# Reset the index, dropping its contents
temps_city.reset_index(drop=True)

Indexes make subsetting simpler. DataFrames have a subsetting method called `.loc[]`, which filters on index values.

In [ ]:
# Index temperatures by city
temps = temperatures.set_index("city")

# Subset on the two cities using .loc[]
temps.loc[["Montreal", "Toronto"]]

You can include multiple columns in the index by passing a list of column names to `.set_index()`.  These are called multi-level indexes, or hierarchical indexes. Subsetting is still possible:

In [ ]:
# Set a multi-level index
temps = temperatures.set_index(["country", "city"])

# Subset rows at the outer level index by passing a list
temps.loc[["Germany", "France"]]

# Subset on inner levels by passing a list of tuples
temps.loc[[("Brazil", "Rio De Janeiro"), ("Pakistan", "Lahore")]]

You can sort a DataFrame by elements in the index using `.sort_index()`:

In [ ]:
# Sort by index values
temps.sort_index()

# Sort by index values at the city level
temps.sort_index(level="city")

# Sort by country in alphabetical order then city in descending alphabetical order
temps.sort_index(level=["country", "city"], ascending=[True, False])

### 3.2 Slicing and subsetting with `.loc[]` and `.iloc[]`

- Slicing lets you select consecutive elements of an object using `first:last` syntax.
- You can only slice an index if the index is sorted (using `.sort_index()`).
- You can also slice columns by passing two arguments to `.loc[]`. The second argument takes column names as the first and last positions to slice on. For example, `df.loc[:, "e":"f"]`.
- Slice multi-index DataFrame by rows with code such as `df.loc[("a", "b"):("c", "d")]`.
- Slice multi-index DataFrames both ways (rows and columns) with code such as `df.loc[("a", "b"):("c", "d"), "e":"f"]`.

In [ ]:
# Set an index and sort
temps = temperatures.set_index(["country", "city"])
temps = temps.sort_index()

# Subset rows from Pakistan to Singapore
temps.loc["Pakistan":"Singapore"]

# Subset rows from Lahore to Rome
temps.loc["Lahore":"Rome"]

# Subset rows from Pakistan, Lahore to Philippines, Manila
temps.loc[("Pakistan", "Lahore"):("Philippines", "Manila")]

# Subset rows and columns at once
temps.loc[("India", "Hyderabad"):("Iraq", "Baghdad"), "date":"avg_temp_c"]

You can slice by dates with the same syntax as other types by:
- Adding the date column to the index, then using `.loc[]` to perform the subsetting. 
- Keeping dates in ISO 8601 format, that is, `"yyyy-mm-dd"` for year-month-day, `"yyyy-mm"` for year-month, and `"yyyy"` for year.

In [ ]:
# Set date as the index and sort the index
temps = temperatures.set_index("date").sort_index()

# Use .loc[] to subset rows in 2010 and 2011
temps.loc["2010":"2011"]

# Use .loc[] to subset rows from Aug 2010 to Feb 2011
temps.loc["2010-08":"2011-02"]

You can slice DataFrames by row and/or column number using the `.iloc[]` method. Here are some examples:

In [ ]:
# Get 25th row, 2nd column (index 25, 1)
temperatures.iloc[25, 1]

# Get the first 5 rows
temperatures.iloc[:5]

# Get columns 3 to 4
temperatures.iloc[:, 2:4]

# Get first 5 rows and columns 3 to 4
temperatures.iloc[:5, 2:4]

### 3.3 Working with pivot tables

Pivot tables are just DataFrames with sorted indexes, so you can apply the methods you've learned so far on them. `.loc[]` is particularly ideal for subsetting pivot tables as shown in the example below:

In [ ]:
# Add a year column to temperatures
temperatures["date"] = pd.to_datetime(temperatures["date"])
temperatures["year"] = temperatures["date"].dt.year

# Pivot avg_temp_c by country and city vs year
temp_by_country_city_vs_year = temperatures.pivot_table(
    "avg_temp_c", index=["country", "city"], columns="year"
)

# Subset for Egypt, Cairo to India, Delhi, and 2005 to 2010
temp_by_country_city_vs_year.loc[("Egypt", "Cairo"):("India", "Delhi"), "2005":"2010"]

## Chapter 4: Creating and visualizing DataFrames 
### 4.1 Visualizing your data
- To display plots, call `plt.show()`.
- To add a legend, call `plt.legend()` and pass a list of labels before `plt.show()`.
- To add a title, use the `title` argument of `plot()`.
- To rotate the x-axis labels, use the `rot` argument of `plot()`. 
Below are some examples of different types of plots.

A **bar plot** reveals relationships between a categorical variable and a numeric variable. 

In [ ]:
# Import matplotlib.pyplot
import matplotlib.pyplot as plt

# Get the total number of avocados sold of each size
nb_sold_by_size = avocado.groupby("size")["nb_sold"].sum()

# Create a bar plot of the number of avocados sold by size
nb_sold_by_size.plot(kind="bar")

# Show the plot
plt.show()

A **line plot** visualizes changes in numeric variables over time. 


In [ ]:
# Get the total number of avocados sold on each date
nb_sold_by_date = avocado.groupby("date")["nb_sold"].sum()

# Create a line plot of the number of avocados sold by date and rotate x-axis labels
nb_sold_by_date.plot(kind="line", rot=30)

# Show the plot
plt.show()

A **scatter plot** visualizes relationships between two numeric variables. 

In [ ]:
# Scatter plot of avg_price vs. nb_sold with title
avocado.plot(x="nb_sold", y="avg_price", kind="scatter", title="# of avocados sold vs. avg price")

# Show the plot
plt.show()

`.hist()`: A **histogram** shows the distribution of a numeric variable. 

- You can use the `bins` argument to adjust the number of bins (or bars). 
- You can also adjust the transparency of the bins using the `alpha` argument. It takes a values from 0 (completely transparent) to 1 (opaque). This is especially useful when layering histograms.

In [ ]:
# Modify bins to 20
avocado[avocado["type"] == "conventional"]["avg_price"].hist(alpha=0.5, bins=20)

# Modify bins to 20
avocado[avocado["type"] == "organic"]["avg_price"].hist(alpha=0.5, bins=20)

# Add a legend
plt.legend(["conventional", "organic"])

# Show the plot
plt.show()

### 4.2 Missing data

Most data is not perfect and often has some pieces missing. That's why it's always a good idea to check for missing values when you get a DataFrame.
- In a pandas DataFrame, missing values are indicated with `NaN`, which stands for "not a number."
- The `.isna()` method returns a Boolean for every single value indicating whether the value is missing or not. 
- Chaining `.isna()` with `.any()` will tell you if there are any missing values in a certain column.
- Chaining `.isna()` with `.sum()` will count the number of `NaN`s in each column.
- You can visualize missing values using `.isna().sum().plot(kind="bar")`.

To practice these methods and concepts, you can use the `avocado_missing` DataFrame which has missing values:

In [ ]:
# Read in the missing avocado data
avocado_missing = pd.read_csv("data/avocado_missing.csv")

# Returns a DataFrame of Booleans
avocado_missing.isna()

# Returns a Series indicating if a column is missing values
avocado_missing.isna().any()

# Returns a Series indicating how many values are missing from a colums
avocado_missing.isna().sum()

# Plot missing values
avocado_missing.isna().sum().plot(kind="bar")
plt.show()

One option is to remove any rows that contain missing values. This is done with `.dropna()`. This is not ideal if you have a lot of missing data, since that means losing a lot of observations.

In [ ]:
# Remove rows with missing values
avocado_complete = avocado_missing.dropna()

Another option is to replace missing values with another value. The `.fillna()` method takes in a value, and replaces all `NaNs` with this value. 

In [ ]:
# Replace missing values with "unknown"
avocado_filled = avocado.fillna("unknown")

### 4.3 Creating DataFrames
A dictionary is a way of storing data. It holds a set of key-value pairs and you can access values through their keys. You can create and access a dictionary like this:

In [ ]:
# Create a dictionary with 3 key-value pairs
my_dict = {"key1": 64, "key2": 57, "key3": 100}

# Access a value via its key with square brackets
my_dict["key2"]

There are many ways to create DataFrames from scratch, but the course covers two essential ways. The first is to use a list of dictionaries. Here, the DataFrame is constructed row by row.

In [ ]:
# Create a list of dictionaries with new data
avocados_list = [
    {"date": "2019-11-03", "small_sold": 10376832, "large_sold": 7835071},
    {"date": "2019-11-10", "small_sold": 10717154, "large_sold": 8561348},
]

# Convert list into DataFrame
avocados_2019 = pd.DataFrame(avocados_list)

# Preview the new DataFrame
avocados_2019

You can also create a DataFrame from a dictionary of lists. Here, the DataFrame is constructed column by column. In this case, the dictionary's keys are column names and the values are a list of column values.

In [ ]:
# Create a dictionary of lists with new data
avocados_dict = {
  "date": ["2019-11-03", "2019-11-10"],
  "small_sold": [10376832, 10717154],
  "large_sold": [7835071, 8561348]
}

# Convert dictionary into DataFrame
avocados_2019 = pd.DataFrame(avocados_dict)

# Preview the new DataFrame
avocados_2019

### 4.4 Reading and writing CSVs

- CSV stands for comma-separated values. It's a common storage file type and designed for DataFrame-like data. Most database and spreadsheet programs can use them or create them.
- In the `data/` folder of this notebook's directory (navigate to the left bar), you can find examples of CSVs used throughout this notebook.
- To put CSV data in a DataFrame, you can use `pd.read_csv("filepath.csv")`. You may have noticed this has been done for you throughout the notebook.
- To convert a DataFrame to a CSV, you can use `pd.to_csv("newfilepath.csv")`.

In [ ]:
# Importing data/avocado.csv as a DataFrame
avocado = pd.read_csv("data/avocado.csv")

# Make some changes, like creating a new column for total sales
avocado["total_sales"] = avocado["avg_price"] * avocado["nb_sold"]

# Save the modified DataFrame as data/avocado_adjusted.csv
avocado.to_csv("data/avocado_adjusted.csv")

After running the cell above, you should find `data/avocado_adjusted.csv` in your notebook's data directory!